## Блок 03 - Первые отчеты по API

In [121]:
import requests
import yaml
from pprint import pprint 

f = open('config.yaml', 'r')
config = yaml.full_load(f)
token = config['yandex_metrika_token']
API_URL = 'https://api-metrika.yandex.net/stat/v1/data'
startDate = '2022-01-01'
endDate = '2022-01-07'
counter = '53600281' #'21075004'
dimensions = ['ym:s:date']
metrics = ['ym:s:visits', 'ym:s:users', 'ym:s:pageviews', 'ym:s:percentNewVisitors'] #, 'ym:s:newUsers']
dimensions_string = ','.join(dimensions)
metrics_string = ','.join(metrics)
sort = dimensions
params = {
    'ids': counter,
    'date1': startDate,
    'date2': endDate,
    'group': 'day',
    'sort': sort,
    'dimensions': dimensions_string,
    'metrics': metrics_string,
    'oauth_token': token
}

In [122]:
r = requests.get(API_URL, params= params)
data = r.json()

In [123]:
pprint(data)

{'contains_sensitive_data': False,
 'data': [{'dimensions': [{'name': '2022-01-01'}],
           'metrics': [541.0, 502.0, 732.0, 88.24701195]},
          {'dimensions': [{'name': '2022-01-02'}],
           'metrics': [686.0, 647.0, 864.0, 86.24420402]},
          {'dimensions': [{'name': '2022-01-03'}],
           'metrics': [827.0, 772.0, 1041.0, 90.67357513]},
          {'dimensions': [{'name': '2022-01-04'}],
           'metrics': [933.0, 880.0, 1171.0, 88.06818182]},
          {'dimensions': [{'name': '2022-01-05'}],
           'metrics': [943.0, 895.0, 1200.0, 88.15642458]},
          {'dimensions': [{'name': '2022-01-06'}],
           'metrics': [998.0, 935.0, 1232.0, 87.70053476]},
          {'dimensions': [{'name': '2022-01-07'}],
           'metrics': [800.0, 770.0, 1034.0, 86.49350649]}],
 'data_lag': 187,
 'max': [998.0, 935.0, 1232.0, 90.67357513],
 'min': [541.0, 502.0, 732.0, 86.24420402],
 'query': {'adfox_event_id': '0',
           'attr_name': '',
           'attribut

In [124]:
for line in data['data']:
    print(line)

{'dimensions': [{'name': '2022-01-01'}], 'metrics': [541.0, 502.0, 732.0, 88.24701195]}
{'dimensions': [{'name': '2022-01-02'}], 'metrics': [686.0, 647.0, 864.0, 86.24420402]}
{'dimensions': [{'name': '2022-01-03'}], 'metrics': [827.0, 772.0, 1041.0, 90.67357513]}
{'dimensions': [{'name': '2022-01-04'}], 'metrics': [933.0, 880.0, 1171.0, 88.06818182]}
{'dimensions': [{'name': '2022-01-05'}], 'metrics': [943.0, 895.0, 1200.0, 88.15642458]}
{'dimensions': [{'name': '2022-01-06'}], 'metrics': [998.0, 935.0, 1232.0, 87.70053476]}
{'dimensions': [{'name': '2022-01-07'}], 'metrics': [800.0, 770.0, 1034.0, 86.49350649]}


In [125]:
# Считаем среднее значение доли новых посетителей по дням
new_visitors = 0
visitors = 0
for line in data['data']:
    visits_date = line['dimensions'][0]['name']
    visits = line['metrics'][0]
    users = line['metrics'][1]
    pageviews = line['metrics'][2]
    new_visitors_in_percents = line['metrics'][3]
    print(visits_date, int(visits), int(users), int(pageviews), round(new_visitors_in_percents, 2))
    new_visitors += round((users * new_visitors_in_percents) / 100)
    visitors += users
print(new_visitors, visitors, round(new_visitors / visitors, 4))

2022-01-01 541 502 732 88.25
2022-01-02 686 647 864 86.24
2022-01-03 827 772 1041 90.67
2022-01-04 933 880 1171 88.07
2022-01-05 943 895 1200 88.16
2022-01-06 998 935 1232 87.7
2022-01-07 800 770 1034 86.49
4751 5401.0 0.8797


In [126]:
# Доля новых пользователей в веб-интерфейсе за всю неделю оказалась заметно выше 
# (значение 92.04% в строке "Итого и средние"), чем за каждый день в отдельности 
# (за 07.01 - 86,49%, за 06.01 - 87,7% итд). Какой результат верен - итоговый или по дням? 
# Ведь не может среднее за неделю быть больше, чем значение за каждый день в отдельности.

# Ответ
# Оба показателя верны, т.к. в недельной статистике "Ежедневные новые пользователи" в течении недели будут
# учитываться только один раз.

In [131]:
# Шаблоны отчётов
import requests
import yaml

f = open('config.yaml', 'r')
config = yaml.full_load(f)
token = config['yandex_metrika_token']
API_URL = 'https://api-metrika.yandex.net/stat/v1/data'
startDate = '2022-01-01'
endDate = '2022-01-07'
counter = '53600281' #'21075004'
sort = dimensions
preset = 'traffic'
params = {
    'ids': counter,
    'date1': startDate,
    'date2': endDate,
    'group': 'day',
    #'sort': sort,
    'preset': preset,
    'oauth_token': token
}
r = requests.get(API_URL, params= params)
data = r.json()

In [135]:
# Количество метрик в пресете
len(data['query']['metrics'])

7

In [164]:
# Фильтры в отчётах
import requests
import yaml

f = open('config.yaml', 'r')
config = yaml.full_load(f)
token = config['yandex_metrika_token']
API_URL = 'https://api-metrika.yandex.net/stat/v1/data'
startDate = '2022-01-01'
endDate = '2022-01-07'
counter = '44147844'
sort = dimensions
preset = 'traffic'

filters = "ym:s:regionCountryName=='Russia'"

params = {
    'ids': counter,
    'date1': startDate,
    'date2': endDate,
    #'group': 'day',
    #'sort': sort,
    'preset': preset,
    'filters': filters,
    'oauth_token': token
}
r = requests.get(API_URL, params= params)
data = r.json()

In [165]:
pprint(data['data'])

[{'dimensions': [{'from': '2022-01-03',
                  'id': '2022-01-03',
                  'name': '2022-01-03 - 2022-01-07',
                  'to': '2022-01-07'}],
  'metrics': [283.0,
              261.0,
              441.0,
              89.65517241,
              35.33568905,
              1.55830389,
              76.54416961]},
 {'dimensions': [{'from': '2022-01-01',
                  'id': '2021-12-27',
                  'name': '2022-01-01 - 2022-01-02',
                  'to': '2022-01-02'}],
  'metrics': [125.0, 124.0, 180.0, 92.74193548, 43.2, 1.44, 51.68]}]


In [170]:
# visits_from_Russia - число визитов из России
visits_from_Russia = data['data'][0]['metrics'][0]
print(visits_from_Russia)

283.0


In [172]:
# Получим общее число посещений, для этого отключим фильтр
params = {
    'ids': counter,
    'date1': startDate,
    'date2': endDate,
    #'group': 'day',
    #'sort': sort,
    'preset': preset,
    #'filters': filters,
    'oauth_token': token
}
r = requests.get(API_URL, params= params)
data = r.json()

In [174]:
visits = data['data'][0]['metrics'][0]
print(visits)

1745.0


In [175]:
# Доля визитов из России
print(round(visits_from_Russia/visits, 2))

0.16


##  Блок 04 - Многостраничные отчеты

In [8]:
import requests
import yaml
from pprint import pprint

# Задаем параметры выгрузки. В качестве измерения будет выступать время визита ym:s:dateTime.
# Из метрик для простоты оставим только визиты ym:s:visits.
f = open('config.yaml', 'r')
config = yaml.full_load(f)
token = config['yandex_metrika_token']
API_URL = 'https://api-metrika.yandex.ru/stat/v1/data'
startDate = '2022-01-10'
endDate = '2022-01-16'
counter = '53600281' #'44147844'
metrics = ['ym:s:visits']
dimensions = ['ym:s:dateTime']

# Стандартные преобразования
metrics_string = ','.join(metrics)
dimensions_string = ','.join(dimensions)
sort = dimensions

params = {
'date1': startDate,
'date2': endDate,
'id': counter,
'dimensions': dimensions_string,
'metrics': metrics_string,
'sort': sort,
'oauth_token': token
}

# Выполняем запрос
r = requests.get(API_URL, params = params)
data = r.json()

In [9]:
pprint(data['data'])

[{'dimensions': [{'name': '2022-01-10 00:03:15'}], 'metrics': [1.0]},
 {'dimensions': [{'name': '2022-01-10 00:07:43'}], 'metrics': [1.0]},
 {'dimensions': [{'name': '2022-01-10 00:08:13'}], 'metrics': [1.0]},
 {'dimensions': [{'name': '2022-01-10 00:10:44'}], 'metrics': [1.0]},
 {'dimensions': [{'name': '2022-01-10 00:11:03'}], 'metrics': [1.0]},
 {'dimensions': [{'name': '2022-01-10 00:11:12'}], 'metrics': [1.0]},
 {'dimensions': [{'name': '2022-01-10 00:11:43'}], 'metrics': [1.0]},
 {'dimensions': [{'name': '2022-01-10 00:13:17'}], 'metrics': [1.0]},
 {'dimensions': [{'name': '2022-01-10 00:14:12'}], 'metrics': [1.0]},
 {'dimensions': [{'name': '2022-01-10 00:15:23'}], 'metrics': [1.0]},
 {'dimensions': [{'name': '2022-01-10 00:15:52'}], 'metrics': [1.0]},
 {'dimensions': [{'name': '2022-01-10 00:16:42'}], 'metrics': [1.0]},
 {'dimensions': [{'name': '2022-01-10 00:18:47'}], 'metrics': [1.0]},
 {'dimensions': [{'name': '2022-01-10 00:19:59'}], 'metrics': [1.0]},
 {'dimensions': [{'n

In [11]:
len(data['data'])

100

In [12]:
# По умолчанию Метрика возвращает первые 100 строк.
# Чтобы получить все данные добавляем параметры limit и offset к запросу.
# Обернем процесс выгрузки в отдельную функцию report_for_offset_value

def report_for_offset_value(offset, n_rows):
    params = {
        'date1': startDate,
        'date2': endDate,
        'id': counter,
        'dimensions': dimensions_string,
        'metrics': metrics_string,
        'sort': sort,
        'offset': offset,
        'limit': n_rows,
        'oauth_token': token
        }
    # Выполняем запрос
    r = requests.get(API_URL, params = params)
    return r.json()

In [13]:
# Потестируем работу функции для первых пяти строк отчета.
offset = 1
n_rows = 5

data = report_for_offset_value(offset, n_rows)
pprint(data)

{'contains_sensitive_data': False,
 'data': [{'dimensions': [{'name': '2022-01-10 00:03:15'}], 'metrics': [1.0]},
          {'dimensions': [{'name': '2022-01-10 00:07:43'}], 'metrics': [1.0]},
          {'dimensions': [{'name': '2022-01-10 00:08:13'}], 'metrics': [1.0]},
          {'dimensions': [{'name': '2022-01-10 00:10:44'}], 'metrics': [1.0]},
          {'dimensions': [{'name': '2022-01-10 00:11:03'}], 'metrics': [1.0]}],
 'data_lag': 215,
 'max': [1.0],
 'min': [1.0],
 'query': {'adfox_event_id': '0',
           'attr_name': '',
           'attribution': 'LastSign',
           'auto_group_size': '1',
           'currency': 'RUB',
           'date1': '2022-01-10',
           'date2': '2022-01-16',
           'dimensions': ['ym:s:dateTime'],
           'group': 'Week',
           'ids': [53600281],
           'limit': 5,
           'metrics': ['ym:s:visits'],
           'offline_window': '21',
           'offset': 1,
           'quantile': '50',
           'sort': ['ym:s:dateTime']

In [16]:
# Выводим значения измерений и метрик

for line in data['data']:
    time_of_visit = line['dimensions'][0]['name']
    visits = line['metrics'][0]
    print(time_of_visit, visits)

2022-01-10 00:03:15 1.0
2022-01-10 00:07:43 1.0
2022-01-10 00:08:13 1.0
2022-01-10 00:10:44 1.0
2022-01-10 00:11:03 1.0


In [28]:
# Пришем цикл который выгружает все строки данного отчёта, для этого сначала вызываем функцию с
# параметрами offset = 1 и n_rows = 1 и достаём из ответа сервера значение total_rows.
# Это значение в цикле будет использовано для вычисления оффсета последнего запроса

data = report_for_offset_value(1, 1)
total_rows = int(data['total_rows'])
print(total_rows)

10513


In [61]:
# Задаём необходимые параметры и заполняем словарь all_data{time_of_visits: visits}
offset = 1
n_rows = 1000
all_data = {}
while offset < total_rows:
    data = report_for_offset_value(offset, n_rows)
    for line in data['data']:
        time_of_visit = line['dimensions'][0]['name']
        visits = line['metrics'][0]
        all_data[time_of_visit] = visits
    if offset + n_rows > total_rows:
        n_rows = total_rows - offset
    offset += n_rows
    print (offset)

1001
2001
3001
4001
5001
6001
7001
8001
9001
10001
10513


In [74]:
# Проверяем длину словаря
len(all_data) == total_rows

True

In [73]:
# Выводим словарь
all_data

{'2022-01-10 00:03:15': 1.0,
 '2022-01-10 00:07:43': 1.0,
 '2022-01-10 00:08:13': 1.0,
 '2022-01-10 00:10:44': 1.0,
 '2022-01-10 00:11:03': 1.0,
 '2022-01-10 00:11:12': 1.0,
 '2022-01-10 00:11:43': 1.0,
 '2022-01-10 00:13:17': 1.0,
 '2022-01-10 00:14:12': 1.0,
 '2022-01-10 00:15:23': 1.0,
 '2022-01-10 00:15:52': 1.0,
 '2022-01-10 00:16:42': 1.0,
 '2022-01-10 00:18:47': 1.0,
 '2022-01-10 00:19:59': 1.0,
 '2022-01-10 00:20:08': 1.0,
 '2022-01-10 00:20:10': 1.0,
 '2022-01-10 00:23:08': 1.0,
 '2022-01-10 00:24:53': 1.0,
 '2022-01-10 00:25:13': 1.0,
 '2022-01-10 00:28:08': 1.0,
 '2022-01-10 00:28:56': 1.0,
 '2022-01-10 00:30:10': 1.0,
 '2022-01-10 00:30:53': 1.0,
 '2022-01-10 00:31:13': 1.0,
 '2022-01-10 00:31:21': 1.0,
 '2022-01-10 00:33:39': 1.0,
 '2022-01-10 00:36:56': 1.0,
 '2022-01-10 00:39:10': 1.0,
 '2022-01-10 00:42:50': 1.0,
 '2022-01-10 00:49:41': 1.0,
 '2022-01-10 00:50:13': 2.0,
 '2022-01-10 00:50:15': 1.0,
 '2022-01-10 00:51:33': 1.0,
 '2022-01-10 00:51:40': 1.0,
 '2022-01-10 0

##  Блок 05 - Обработка произвольного отчёта Метрики

In [84]:
import requests
import yaml
from pprint import pprint

f = open('config.yaml', 'r')
config = yaml.full_load(f)
token = config['yandex_metrika_token']
API_URL = 'https://api-metrika.yandex.ru/stat/v1/data'
startDate = '2022-01-10'
endDate = '2022-01-16'
counter = '53600281' #'44147844'
dimensions = ['ym:s:date', 'ym:s:deviceCategory', 'ym:s:browser']
metrics = ['ym:s:visits', 'ym:s:users', 'ym:s:pageviews']

# Стандартные преобразования
metrics_string = ','.join(metrics)
dimensions_string = ','.join(dimensions)
sort = dimensions

params = {
'date1': startDate,
'date2': endDate,
'id': counter,
'dimensions': dimensions_string,
'metrics': metrics_string,
'sort': sort,
'oauth_token': token
}

# Выполняем запрос
r = requests.get(API_URL, params = params)
data = r.json()

In [85]:
# Запрос к API Яндекс.Метрики. Возвращает JSON ответа
def make_request(**kwargs):
    return requests.get(API_URL, params = kwargs).json()

In [86]:
data = make_request(date1 = startDate, date2 = endDate, dimensions = ','.join(dimensions),
                    metrics = ','.join(metrics), id = counter, oauth_token = token, accurancy = 1)

In [88]:
data['data']

[{'dimensions': [{'name': '2022-01-11'},
   {'icon_id': 'desktop',
    'icon_type': 'device',
    'name': 'ПК',
    'id': 'desktop'},
   {'icon_id': '6',
    'icon_type': 'browser',
    'name': 'Google Chrome',
    'id': '6'}],
  'metrics': [871.0, 809.0, 1095.0]},
 {'dimensions': [{'name': '2022-01-12'},
   {'icon_id': 'desktop',
    'icon_type': 'device',
    'name': 'ПК',
    'id': 'desktop'},
   {'icon_id': '6',
    'icon_type': 'browser',
    'name': 'Google Chrome',
    'id': '6'}],
  'metrics': [849.0, 807.0, 1122.0]},
 {'dimensions': [{'name': '2022-01-13'},
   {'icon_id': 'desktop',
    'icon_type': 'device',
    'name': 'ПК',
    'id': 'desktop'},
   {'icon_id': '6',
    'icon_type': 'browser',
    'name': 'Google Chrome',
    'id': '6'}],
  'metrics': [846.0, 779.0, 1143.0]},
 {'dimensions': [{'name': '2022-01-14'},
   {'icon_id': 'desktop',
    'icon_type': 'device',
    'name': 'ПК',
    'id': 'desktop'},
   {'icon_id': '6',
    'icon_type': 'browser',
    'name': 'Google 

In [89]:
len(data['data'])

100

In [90]:
# Выводим лист измерений
for line in data['data']:
    print(line['dimensions'])
    break

[{'name': '2022-01-11'}, {'icon_id': 'desktop', 'icon_type': 'device', 'name': 'ПК', 'id': 'desktop'}, {'icon_id': '6', 'icon_type': 'browser', 'name': 'Google Chrome', 'id': '6'}]


In [91]:
# Выводим все значения измерений для каждого ключа 'name'
for line in data['data']:
    for dimensions_value in line['dimensions']:
        print(dimensions_value['name'])
    break

2022-01-11
ПК
Google Chrome


In [93]:
# Более короткая запись
for line in data['data']:
    print([x['name'] for x in line['dimensions']])
    break

['2022-01-11', 'ПК', 'Google Chrome']


In [94]:
# Добавляем аналогичный цикл для значений метрик
for line in data['data']:
    print(line['metrics'])
    break

[871.0, 809.0, 1095.0]


In [95]:
# Объединяем измерения и метрики каждой строчки в лист 'report'
report = []
for line in data['data']:
    dimensions_list = [x['name'] for x in line['dimensions']]
    metrics_list = line['metrics']
    report.append(dimensions_list + metrics_list)
    print(report)
    break

[['2022-01-11', 'ПК', 'Google Chrome', 871.0, 809.0, 1095.0]]


In [96]:
# Оформим цикл в функцию
def json_handler(data):
    report = []
    for line in data['data']:
        dimensions_list = [x['name'] for x in line['dimensions']]
        metrics_list = line['metrics']
        report.append(dimensions_list + metrics_list)
    return report

In [97]:
json_handler(data)

[['2022-01-11', 'ПК', 'Google Chrome', 871.0, 809.0, 1095.0],
 ['2022-01-12', 'ПК', 'Google Chrome', 849.0, 807.0, 1122.0],
 ['2022-01-13', 'ПК', 'Google Chrome', 846.0, 779.0, 1143.0],
 ['2022-01-14', 'ПК', 'Google Chrome', 805.0, 762.0, 925.0],
 ['2022-01-10', 'ПК', 'Google Chrome', 796.0, 723.0, 1005.0],
 ['2022-01-15', 'ПК', 'Google Chrome', 373.0, 358.0, 484.0],
 ['2022-01-16', 'ПК', 'Google Chrome', 347.0, 321.0, 468.0],
 ['2022-01-11', 'Смартфоны', 'Chrome Mobile', 250.0, 229.0, 328.0],
 ['2022-01-13', 'Смартфоны', 'Chrome Mobile', 230.0, 213.0, 277.0],
 ['2022-01-11', 'ПК', 'Яндекс.Браузер', 219.0, 207.0, 269.0],
 ['2022-01-12', 'ПК', 'Яндекс.Браузер', 209.0, 196.0, 250.0],
 ['2022-01-14', 'Смартфоны', 'Chrome Mobile', 206.0, 192.0, 261.0],
 ['2022-01-15', 'Смартфоны', 'Chrome Mobile', 194.0, 182.0, 241.0],
 ['2022-01-12', 'Смартфоны', 'Chrome Mobile', 192.0, 183.0, 244.0],
 ['2022-01-16', 'Смартфоны', 'Chrome Mobile', 181.0, 168.0, 236.0],
 ['2022-01-14', 'ПК', 'Яндекс.Браузер

In [102]:
# Посчитать долю блокировщиков по источникам трафика
# ym:s:hasAdBlocker, ym:s:TrafficSource, ym:s:visits
import requests
import yaml
from pprint import pprint

f = open('config.yaml', 'r')
config = yaml.full_load(f)
token = config['yandex_metrika_token']
API_URL = 'https://api-metrika.yandex.ru/stat/v1/data'
startDate = '2022-01-10'
endDate = '2022-01-16'
counter = '53600281' #'44147844'
dimensions = ['ym:s:hasAdBlocker', 'ym:s:<attribution>TrafficSource']
metrics = ['ym:s:visits']

# Стандартные преобразования
metrics_string = ','.join(metrics)
dimensions_string = ','.join(dimensions)
sort = dimensions

params = {
'date1': startDate,
'date2': endDate,
'id': counter,
'dimensions': dimensions_string,
'metrics': metrics_string,
'sort': sort,
'oauth_token': token
}

# Выполняем запрос
r = requests.get(API_URL, params = params)
data = r.json()

In [103]:
data

{'query': {'ids': [53600281],
  'dimensions': ['ym:s:hasAdBlocker', 'ym:s:lastSignTrafficSource'],
  'metrics': ['ym:s:visits'],
  'sort': ['ym:s:hasAdBlocker', 'ym:s:lastSignTrafficSource'],
  'date1': '2022-01-10',
  'date2': '2022-01-16',
  'limit': 100,
  'offset': 1,
  'group': 'Week',
  'auto_group_size': '1',
  'attr_name': '',
  'quantile': '50',
  'offline_window': '21',
  'attribution': 'LastSign',
  'currency': 'RUB',
  'adfox_event_id': '0'},
 'data': [{'dimensions': [{'name': 'Блокировщик используется', 'id': 'yes'},
    {'icon_id': '0',
     'icon_type': 'traffic-source',
     'name': 'Прямые заходы',
     'id': 'direct'}],
   'metrics': [18.0]},
  {'dimensions': [{'name': 'Блокировщик используется', 'id': 'yes'},
    {'icon_id': '1',
     'icon_type': 'traffic-source',
     'name': 'Переходы по ссылкам на сайтах',
     'id': 'referral'}],
   'metrics': [46.0]},
  {'dimensions': [{'name': 'Блокировщик используется', 'id': 'yes'},
    {'icon_id': '2',
     'icon_type': 'tr

In [171]:
report = {}
for line in data['data']:
    tmp = {}
    if line['dimensions'][0]['id'] == 'yes':
        tmp['yes'] = line['metrics']
        print(tmp)
        report[line['dimensions'][1]['name']] = tmp
        print(report)
        #print(1, line['dimensions'][1]['name'], line['metrics'])
    if line['dimensions'][0]['id'] == 'no':
        tmp[line['dimensions'][0]['id']] = line['metrics']
        print(tmp)
        print(line['dimensions'][1]['name'])
        # report[line['dimensions'][1]['name']].update({, tmp})
        print(report)
        #print(0, line['dimensions'][1]['name'], line['metrics'])

{'yes': [18.0]}
{'Прямые заходы': {'yes': [18.0]}}
{'yes': [46.0]}
{'Прямые заходы': {'yes': [18.0]}, 'Переходы по ссылкам на сайтах': {'yes': [46.0]}}
{'yes': [420.0]}
{'Прямые заходы': {'yes': [18.0]}, 'Переходы по ссылкам на сайтах': {'yes': [46.0]}, 'Переходы из поисковых систем': {'yes': [420.0]}}
{'yes': [3.0]}
{'Прямые заходы': {'yes': [18.0]}, 'Переходы по ссылкам на сайтах': {'yes': [46.0]}, 'Переходы из поисковых систем': {'yes': [420.0]}, 'Переходы из социальных сетей': {'yes': [3.0]}}
{'no': [16.0]}
Внутренние переходы
{'Прямые заходы': {'yes': [18.0]}, 'Переходы по ссылкам на сайтах': {'yes': [46.0]}, 'Переходы из поисковых систем': {'yes': [420.0]}, 'Переходы из социальных сетей': {'yes': [3.0]}}
{'no': [157.0]}
Прямые заходы
{'Прямые заходы': {'yes': [18.0]}, 'Переходы по ссылкам на сайтах': {'yes': [46.0]}, 'Переходы из поисковых систем': {'yes': [420.0]}, 'Переходы из социальных сетей': {'yes': [3.0]}}
{'no': [191.0]}
Переходы по ссылкам на сайтах
{'Прямые заходы': {'y

In [157]:
report

{'Прямые заходы': {'yes': [18.0]},
 'Переходы по ссылкам на сайтах': {'yes': [46.0]},
 'Переходы из поисковых систем': {'yes': [420.0]},
 'Переходы из социальных сетей': {'yes': [3.0]}}

In [167]:
type(report['Прямые заходы'])

dict